In [34]:
# importar módulos
import numpy as np
import qiskit

# Computação quântica
## Conceitos básicos 

- bits quânticos (qbit) 
- operadores sobre um qbit
- Medição
- Simulação utilizando qiskit
- Execução em um processador quântico
- Produto tensorial
- Emaranhamento

## Introdução

* A matéria em escala subatômica apresenta um comportamento diferente do descrito na física clássica. 
    * A computação clássica processa bits que podem estar no "estado" 0 ou 1. 
    * A computação quântica processa bits quânticos ou **qbits** podem estar em um número contínuo de possibilidades. 
    

## Introdução

* Não é claro se existe um ganho computacional ao utilizarmos a computação quântica. No entanto existe uma forte evidência deste ganho.
    * Por exemplo, o algoritmo quântico de fatoração em tempo polinomial.
    * Algoritmo de busca com custo $O(\sqrt{n})$

## Introdução

O principal objetivo deste curso é apresentar os conceitos da computação quântica, os principais algoritmos e métodos de programação para processadores quânticos de pequena escala.

Diversas empresas, universidades e países estão realizando investimentos para a construção de um computador quântico. Apesar dos resultados teóricos, até a data desta aula ainda não existe uma aplicação prática de computadores quânticos que não possa ser realizada por computadores clássicos.

## Bits quânticos
* Na literatura de computação quântica a notação de dirac $|\cdot \rangle$ é utilizada para representar vetores. Por exemplo,

$$|0\rangle = \begin{bmatrix}1 \\ 0\end{bmatrix} \mbox{ e } |1\rangle = \begin{bmatrix}0 \\ 1\end{bmatrix}.$$


## Bits quânticos
* Enquanto os bits podem estar em apenas dois possíveis estados, os bits quânticos estão no espaço vetorial $\mathbb{C}^2$ e podem estar em uma superposição de estados (em uma combinação linear de $|0\rangle$ e $|1\rangle$) $$|\psi\rangle = \alpha|0\rangle + \beta|1\rangle,$$.

    onde $\alpha$ e $\beta$ são denominados amplitudes de probabilidade e $|\alpha|^2+|\beta|^2 = 1$

## Medição projetiva

* Os valores das amplitudes de probabilidade de um qbit não podem ser recuperados. 
* A única informação que obtemos ao observar um qbit $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$ é 0 com probabilidade $|\alpha|^2$ e 1 com probabilidade $|\beta|^2$. 
* Após a medição o estado do qbit colapsa e sucessivas medições retornarão o mesmo resultado.

# Medição projetiva (exemplo)
* Com que probabilidade será obtido o resultado 0 ao medir o estado $|\psi\rangle = \frac{1}{\sqrt{3}}|0\rangle + \sqrt{\frac{2}{3}}|1\rangle$? 


Ao realizar uma medição será obtido 0 com probabilidade $\frac{1}{3}$ e neste caso o qbit irá colapsar para o estado $|\psi\rangle =|0\rangle$ ou 1 com probabilidade $\frac{2}{3}$ e neste caso o qbit irá colapsar para o estado $|\psi\rangle =|1\rangle$.

## Operadores quânticos sobre um qbit
* Na base computacional (ou base canônica) $\{|0\rangle, |1\rangle\}$, um operador quântico sobre um qbit pode ser representado por uma matriz unitária $2 \times 2$. 

* Uma matriz $U$ é unitária se ao ser multiplicada por sua transposta conjugada $U^\dagger$ obtiver a matriz identidade como resultado $UU^\dagger = I$. 

* Toda matriz unitária $2 \times 2$ representa um operador quântico válido.



* Alguns exemplos de operadores quânticos $I$, $H$, $X$, $Z$, $S$, $T$

$$I = \begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}\mbox{, } 
%
X = \begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix}\mbox{, }
%
H = \frac{1}{\sqrt{2}}\begin{bmatrix}
1 & 1 \\
1 & -1
\end{bmatrix}\mbox{, }
Z = \begin{bmatrix}
1 & 0 \\
0 & -1
\end{bmatrix}$$

$$
S = \begin{bmatrix}
1 & 0 \\
0 & i
\end{bmatrix}\mbox{, } 
%
T = \begin{bmatrix}
1 & 0 \\
0 & e^{i\pi/4}
\end{bmatrix}\mbox{, } 
$$

## Operadores quânticos sobre um qbit

* Os operadores de rotação $R_z$, $R_y$, $R_x$ são descritos abaixo.

$$R_x(\theta) = \begin{bmatrix}
cos(\theta/2) & -isen(\theta/2) \\
-isen(\theta/2) & cos(\theta/2)
\end{bmatrix}\mbox{, }
%
R_y(\theta) = \begin{bmatrix}
cos(\theta/2) & -sen(\theta/2) \\
sen(\theta/2) & cos(\theta/2)
\end{bmatrix}
$$

$$
R_z(\theta) = \begin{bmatrix}
e^{-i\theta/2} & 0 \\
0 & e^{i\theta/2}
\end{bmatrix}
$$



* Os dispositivos quânticos disponíveis possuem um conjunto finito de funções para representar operadores quânticos. 
* Por exemplo, no computador ibmqx4 estão disponíveis as seguintes operações sobre um qbit.

$$u3(\theta, \phi, \lambda) = 
\begin{bmatrix} 
    cos(\theta/2) & -e^{i\lambda}sen(\theta/2) \\
    e^{i\phi}sen(\theta/2) & e^{i(\lambda+\phi)}cos(\theta/2)
\end{bmatrix}\mbox{, } 
u2(\phi, \lambda)=
\begin{bmatrix}
1 & -e^{i\lambda}\\
e^{i\phi} & e^{i\phi + i\lambda}
\end{bmatrix}
$$

$$u1(\lambda)=
\begin{bmatrix}
1 & 0\\
0 & e^{i\lambda}
\end{bmatrix}$$

## Simulação utilizando o numpy (opcional)

* Podemos pensar em um bit quântico como uma classe com um atributo vetor privado. 
* O vetor é inicializado em $|0\rangle$ e só pode ser modificado pela ação de operadores quânticos e medições.

In [35]:
import numpy as np
import random
class qbit:
    def __init__(self):
        self._vector = np.matrix([[1],[0]]) # _ indica aproximadamente variável protegida
    def measure(self):
        a = random.random()
        if a < self._vector[0]**2:
            self._vector = np.matrix([[1],[0]])
            return 0
        else:
            self._vector = np.matrix([[0],[1]])
            return 1
    def __str__(self):
        return str(self._vector[0]) + '|0>' + ' + ' + str(self._vector[1]) + '|1>'
    


In [36]:
class qop:
    def __init__(self, matrix):
        self.matrix = np.matrix(matrix)
    def __mul__(self, qbit):
        qbit._vector = self.matrix * qbit._vector
    
    def __str__(self):
        return str(self.matrix)

In [37]:
# inicializa um qbit
psi = qbit() 

# inicializa um operador quântico
a = (1/np.sqrt(2)) * np.matrix([[1, 1],[1, -1]])
H = qop(a)

#Aplica o operador ao bit quântico
H * psi

In [38]:
print(psi) # esta operação não é possível em qbit real

[[0.70710678]]|0> + [[0.70710678]]|1>


In [39]:
# Medição do qbit
psi.measure()

1

In [40]:
# Estado após a medição
print(psi) # esta operação não é possível em qbit real

[[0]]|0> + [[1]]|1>


## Qiskit
* O [qiskit](https://qiskit.org) fornece ferramentas para a programação de computadores quânticos na linguagem de programação python. 
* Para inicializar um qbit utilizamos a função ```QuantumRegister``` 
* Para inicializar um bit clássico utilizamos a função ```ClassicalRegister``` 
* É necessário construir um circuito quântico com a classe `QuantumCircuit` para realizar operações sobre os qbits. 
* Estes são os objetos básicos para programar um sistema quântico.


```python 
QuantumRegister(size, name=None) 
# implementa um registrador quântico como um array de qbits

ClassicalRegister(size, name=None) 
# implementa um registrador clássico como um array de bits

QuantumCircuit(*regs, name=None) 
# Cria um circuito quântico. *regs lista de ClassicalRegisters e Quantum registers e name string
```

## Qiskit
* No código abaixo um registrador quântico `qr`, um registrador clássico `cr` e um circuito quântico `qcircuit` contendo `qr` e `cr` são criados. 
* Em seguida o operador Hadammard `qcircuit.h(q)` é aplicado no registrador `q` e uma medição do registrador `qr` é realizada e seu resultado é armazenado no registrador `cr`.

In [41]:
qr = qiskit.QuantumRegister(1, 'q')
cr = qiskit.ClassicalRegister(1, 'c')
qcircuit = qiskit.QuantumCircuit(qr, cr) 
qcircuit.h(qr)
qcircuit.measure(qr, cr)

## Circuito

* Podemos visualizar o circuito `qcircuit` utilizando a função `draw()` da classe `QuantumCircuit`. 
* Um circuito é lido da esquerda para direita. 
* No circuito abaixo o operador $H$ é aplicado para em seguida realizarmos uma medição.

In [42]:
qcircuit.draw()


┌───┐┌─┐
q_0: ┤ H ├┤M├
     └───┘└╥┘
c_0: ══════╩═

## qasm_simulator
* Para executar um circuito quântico devemos utilizar o módulo `Aer`. 
* A função `Aer.get_backend` permite selecionar onde o circuito será executado.
* A função `qiskit.execute` permite executar o circuito.


In [44]:
from qiskit import Aer
backend = Aer.get_backend('qasm_simulator')
job = qiskit.execute(qcircuit, backend, shots=1024)

In [45]:
result = job.result()
counts = result.get_counts(qcircuit)
print(counts)

{'1': 506, '0': 518}


## Executando o circuito em um processador quântico

* A IBM disponibiliza protótipos de processadores quânticos com 
    * 5 qbits (ibmq_5_yorktown - ibmqx2 e ibmq_5_tenerife - ibmqx4) 
    * 14 qbits (ibmq_16_melbourne)
* Para acessar estes computadores é necessário criar uma conta em https://quantum-computing.ibm.com/login.

* Após criar uma conta é necessário acessar perfil $\rightarrow$ my account $\rightarrow$ copy token. Esse token é pessoal e não deve ser compartilhado. Ao executar a célula abaixo informe seu token quando solicitado para registrar este notebook.

In [23]:
from qiskit import IBMQ 
import getpass

MY_API_TOKEN = getpass.getpass("Informe seu API_TOKEN: ")

Informe seu API_TOKEN: ········


In [24]:
IBMQ.enable_account(MY_API_TOKEN)
provider = IBMQ.get_provider()
provider.backends(operational=True)

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_athens') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_rome') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [15]:
qcircuit.draw()

┌───┐┌─┐
q_0: ┤ H ├┤M├
     └───┘└╥┘
c_0: ══════╩═

In [26]:
qbackend = provider.get_backend('ibmqx2')
qjob = qiskit.execute(qcircuit, qbackend, shots=1024)
qresult = qjob.result()
qcounts = qresult.get_counts(qcircuit)
print(qcounts)

{'1': 552, '0': 472}


## Exemplo
Crie um circuito que prepare o estado quântico $\sqrt{0.1}|0\rangle + \sqrt{0.9}|1\rangle$

In [46]:
desired_state = [np.sqrt(0.1), np.sqrt(0.9)]
theta = 2 * np.arccos(desired_state[0])

qc = QuantumCircuit(1)
qc.ry(theta, 0)

backend = qiskit.Aer.get_backend("statevector_simulator")
result = qiskit.execute(qc, backend).result()
state_vector = result.get_statevector()
print("state_vector", state_vector)
print("desired_state", desired_state)


state_vector [0.31622777+0.j 0.9486833 +0.j]
desired_state [0.31622776601683794, 0.9486832980505138]


## Exercícios


1. Mostre que o operador $H$ é unitário.

2. Mostre que $H^2 = I$


3. Quais são os autovalores e autovetores de $H$?

4. Determine o estado do qbit $|\psi\rangle = ZHX|0\rangle$. Verifique a probabilidade de obtermos 0 ou 1 ao realizar a medição do qbit e simule as operações utilizando o qiskit para verificar sua estimativa.



5. Implemente um circuito quântico que crie a partir do $|0\rangle$ o estado $\frac{1}{\sqrt{7}}|0\rangle - \frac{\sqrt{6}}{\sqrt{7}}|1\rangle$



6. Utilizando o numpy, mostre que $HXH = Z$, $HYH = -Y$ e $HZH=X$

## Múltiplos qbits

* O produto tensorial $\otimes$ é utilizado para representar múltiplos qbits.
* Fundamental para compreensão da computação quântica.
* No python, o produto tensorial é implementado na função `np.kron`

## Produto tensorial

Seja $A$ uma matriz $m \times n$ e $B$ uma matriz $p \times q$. O produto de Kronecker será a matriz

$$A \otimes B =
\begin{pmatrix}
A_{11}B & A_{12}B & \cdots & A_{1n}B \\
A_{21}B & A_{22}B & \cdots & A_{2n}B \\
\vdots & \vdots & \vdots & \vdots \\
A_{m1}B & A_{m2}B & \cdots & A_{mn}B
\end{pmatrix}
$$

# Produto tensorial

* Sejam $|\psi_0\rangle = |0\rangle$ e $|\psi_1\rangle = |1\rangle$
* Calculando o produto tensorial obtemos $|\psi_0\rangle \otimes |\psi_0\rangle$, $|\psi_0\rangle \otimes |\psi_1\rangle$, $|\psi_1\rangle \otimes |\psi_0\rangle$ e $|\psi_1\rangle \otimes |\psi_1\rangle$

$|0\rangle \otimes |0\rangle = |00\rangle$

In [47]:
psi0 = np.array([1, 0]).T
psi1 = np.array([0, 1]).T

In [20]:
psi00 = np.kron(psi0, psi0)
psi01 = np.kron(psi0, psi1)
psi10 = np.kron(psi1, psi0)
psi11 = np.kron(psi1, psi1)
print([psi00.T, psi01.T, psi10.T, psi11.T])

[array([1, 0, 0, 0]), array([0, 1, 0, 0]), array([0, 0, 1, 0]), array([0, 0, 0, 1])]


In [8]:
H = (1/np.sqrt(2)) * np.array([[1, 1],[1, -1]])
print(H, "\n")

print("$H \otimes H =$")
print(np.kron(H,H))

[[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]] 

$H \otimes H =$
[[ 0.5  0.5  0.5  0.5]
 [ 0.5 -0.5  0.5 -0.5]
 [ 0.5  0.5 -0.5 -0.5]
 [ 0.5 -0.5 -0.5  0.5]]


## Produto tensorial

* Sejam $V$ e $W$ espaços vetoriais de dimensão $m$ e $n$ e com bases $A = \{|\psi_1\rangle, \cdots |\psi_m\rangle\}$ e $B = \{|\theta_1\rangle, \cdots |\theta_m\rangle\}$
* O espaço vetorial $V \otimes W$ é o espaço cuja base é formada pelos $mn$ elementos da forma $|\alpha_i\rangle \otimes |\beta_j\rangle$.

* O produto tensorial segue as seguintes propriedades:
    * $(|v_1\rangle+|v_2\rangle) \otimes |w\rangle = |v_1\rangle\otimes|w\rangle + |v_2\rangle\otimes|w\rangle$
    * $|v\rangle \otimes (|w_1\rangle+|w_2\rangle) = |v\rangle\otimes|w_1\rangle + |v\rangle\otimes|w_2\rangle$
    * $(a|v\rangle) \otimes |w\rangle = |v\rangle \otimes (a|w\rangle) = a(|v\rangle \otimes |w\rangle)$

* Os elementos de $V \otimes W$ podem ser escritos como
$$\alpha_{11} (|\psi_1\rangle\otimes|\phi_1\rangle) + \cdots + \alpha_{mn}|\psi_m\rangle \otimes |\theta_n\rangle$$
* Escrevemos $|v\rangle|w\rangle$ ou $|vw\rangle$ para denotar $|v\rangle \otimes |w\rangle$.

# Produto tensorial
* Sejam $A$ e $B$ são operadores lineares sobre $V$ e $W$ e $|v\rangle$ e $|w\rangle$ são vetores em $V$ e $W$,  $A\otimes B$ é definido como

$$(A\otimes B)(|v\rangle \otimes |w\rangle) = A|v\rangle \otimes B |w\rangle$$

### Exercício
 * Calcule: 
     * $(H \otimes H) (|0\rangle \otimes |0\rangle)$
     * $H|0\rangle \otimes H|0\rangle$

In [10]:
from qiskit import QuantumCircuit
qc = QuantumCircuit(2)
qc.h(0)
qc.h(1)
qc.draw()

┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     └───┘

## Porta não controlada

* A porta CNOT atua sobre dois qbits e inverte o segundo caso o primeiro esteja no estado $|1\rangle$
    * $CNOT |00\rangle = |00\rangle$
    * $CNOT |01\rangle = |01\rangle$
    * $CNOT |10\rangle = |11\rangle$
    * $CNOT |11\rangle = |10\rangle$


### Exemplo
* Determine a matriz do operador CNOT na base $\{|00\rangle, |01\rangle, |10\rangle, |11\rangle\}$

In [48]:
qc = QuantumCircuit(2, 2) #inicializa um circuito com 2 qbits e 2 bits
qc.h(0) # prepara o estado 1/\sqrt{2} (|00> + |01>)

qc.cx(0, 1) 

qc.measure(0, 0)
qc.measure(1, 1)

backend = qiskit.Aer.get_backend("qasm_simulator")

result = qiskit.execute(qc, backend, shots=1024).result()
counts = result.get_counts(qc)
print(counts)

{'11': 516, '00': 508}


## Estados emaranhados

* Determine o estado $|\psi\rangle = CNOT(H\otimes I)|00\rangle$
* Escreva este estado como o produto tensorial de dois qubits na forma $(a_0|0\rangle+b_0|1\rangle) \otimes (a_1|0\rangle+b_1|1\rangle)$


* Estados com $n$ qbits que não podem ser fatorados como um produto tensorial de $n$ qbits são denominados **emaranhados**

## Medição de sistemas com múltiplos qbits
* Seja $|\psi\rangle = a_{00}|00\rangle + a_{01}|01\rangle + a_{10}|10\rangle + a_{11}|11\rangle$
* Que resultado obtemos ao medir os dois qbits?
* E se medirmos apenas o primeiro qbit?


## Exercícios

1. Crie um circuito quântico que gere o estado $\sqrt{0.2}|0\rangle + \sqrt{0.8}e^{0.3i}|1\rangle$

In [ ]:
backend = Aer.get_backend("statevector_simulator")
qc = qiskit.QuantumCircuit(1)

# seu código aqui


job = qiskit.execute(qc, backend, shots=1024)
result = job.result()
state_vector = result.get_statevector()
print(state_vector)


2. Crie um circuit quântico que gere o estado $\frac{1}{\sqrt{2}}\left(|00\rangle + |11\rangle\right)$

In [17]:
backend = qiskit.Aer.get_backend("statevector_simulator")
qc = qiskit.QuantumCircuit(2)

# seu código aqui


job = qiskit.execute(qc, backend, shots=1024)
result = job.result()
state_vector = result.get_statevector()
print(state_vector)

[1.+0.j 0.+0.j 0.+0.j 0.+0.j]


3. Crie um circuit quântico que gere o estado $\frac{1}{\sqrt{2}}\left(|00\rangle - e^{0.3\pi}|11\rangle\right)$

In [ ]:
backend = qiskit.Aer.get_backend("statevector_simulator")
qc = qiskit.QuantumCircuit(2)

# seu código aqui


job = qiskit.execute(qc, backend, shots=1024)
result = job.result()
state_vector = result.get_statevector()
print(state_vector)

## Referências
- Yanofsky, Noson S., and Mirco A. Mannucci. Quantum computing for computer scientists. Cambridge University Press, 2008.
- Rieffel, Eleanor G., and Wolfgang H. Polak. Quantum computing: A gentle introduction. MIT Press, 2011.
- Nielsen, Michael A., and Isaac Chuang. "Quantum computation and quantum information." (2002): 558-559.
- https://qiskit.org/
